In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from data_loader import load_data

In [ ]:
data_x, data_y = load_data("data_fill/train_data.csv")
print('feature len:', len([_ for _ in data_x.columns.tolist() ]))

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}
model = LogisticRegression(penalty='l2', max_iter=1000)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(data_x, data_y)
best_model = grid_search.best_estimator_
mean_valid = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]

print("best param: ", grid_search.best_params_)
print(f'cv validation accuracy: {mean_valid:.4f}')

# fit the whole data
best_model.fit(data_x, data_y)

feature len: 156
best param:  {'C': 0.001}
cv validation accuracy: 0.5597


LogisticRegression(C=0.001, max_iter=1000)

In [11]:
test_x = load_data("data_fill/task1_test_data.csv", is_train=False)
test_y_pred = best_model.predict(test_x)
test_sub = pd.DataFrame({
    'id': range(len(test_y_pred)),  # ID from 0 to len(predictions) - 1
    'home_team_win': test_y_pred
})
test_sub.to_csv("logistic_task_1_submission.csv", index=False)

## LGBM

try lgbm with new data loader

In [42]:
import lightgbm as lgb
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

dummy_col = ["home_team_abbr", "away_team_abbr", "season"]
dummy_col = None
data_x, data_y = load_data("data_fill/train_data.csv", dummy_col)
print('feature len:', len([_ for _ in data_x.columns.tolist() ]))

train_x, valid_x, train_y, valid_y = train_test_split(data_x, data_y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [20, 31],
    'max_depth': [-1, 10],
    'min_child_samples': [5, 10]
}

best_accuracy = 0
best_params = None

for params in tqdm(ParameterGrid(param_grid), desc="Tuning parameters"):
    model = lgb.LGBMClassifier(verbose=-1, **params)
    model.fit(train_x, train_y)
    
    valid_y_pred = model.predict(valid_x)
    accuracy = accuracy_score(valid_y, valid_y_pred)
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

print(f"Best parameters: {best_params}")
print(f"Best validation accuracy: {best_accuracy:.4f}")

model = lgb.LGBMClassifier(verbose=-1, **best_params)
model.fit(data_x, data_y)

feature len: 156


Tuning parameters: 100%|██████████| 72/72 [01:00<00:00,  1.18it/s]


Best parameters: {'learning_rate': 0.01, 'max_depth': -1, 'min_child_samples': 5, 'n_estimators': 100, 'num_leaves': 31}
Best validation accuracy: 0.5759


LGBMClassifier(learning_rate=0.01, min_child_samples=5, verbose=-1)

In [43]:
test_x = load_data("data_fill/task1_test_data.csv", is_train=False, dummy_col=dummy_col)
print('feature len:', len([_ for _ in test_x.columns.tolist() ]))

test_y_pred = model.predict(test_x)
test_sub = pd.DataFrame({
    'id': range(len(test_y_pred)),  # ID from 0 to len(predictions) - 1
    'home_team_win': test_y_pred
})
test_sub.to_csv("lgbm_task_1_submission.csv", index=False)

feature len: 156


## Random forest

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid, train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import pandas as pd

# Assuming load_data function is defined elsewhere and works like this:
# def load_data(file_path, dummy_col=None):
#     # Load the data and drop dummy columns if specified
#     ...

# Load data
# dummy_col = ["home_team_abbr", "away_team_abbr", "season"]
dummy_col = None
data_x, data_y = load_data("data_fill/train_data.csv", dummy_col)
print('Feature count:', len(data_x.columns))

# Split data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(data_x, data_y, test_size=0.2, random_state=42)

# Define parameter grid for RandomForestClassifier
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize variables to store the best accuracy and parameters
best_accuracy = 0
best_params = None

# Grid search with tqdm progress bar
for params in tqdm(ParameterGrid(param_grid), desc="Tuning parameters"):
    model = RandomForestClassifier(random_state=42, **params)
    model.fit(train_x, train_y)
    
    valid_y_pred = model.predict(valid_x)
    accuracy = accuracy_score(valid_y, valid_y_pred)
    
    # Update best model if current model's accuracy is better
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

print(f"Best parameters: {best_params}")
print(f"Best validation accuracy: {best_accuracy:.4f}")

# Train final model on the full dataset with the best parameters
final_model = RandomForestClassifier(random_state=42, **best_params)
final_model.fit(data_x, data_y)

Feature count: 156


Tuning parameters: 100%|██████████| 81/81 [09:13<00:00,  6.83s/it]


Best parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best validation accuracy: 0.5727


RandomForestClassifier(max_depth=10, min_samples_leaf=4, random_state=42)

In [ ]:
test_x = load_data("data_fill/task1_test_data.csv", is_train=False, dummy_col=dummy_col)
print('feature len:', len([_ for _ in test_x.columns.tolist() ]))

test_y_pred = final_model.predict(test_x)
test_sub = pd.DataFrame({
    'id': range(len(test_y_pred)),  # ID from 0 to len(predictions) - 1
    'home_team_win': test_y_pred
})
test_sub.to_csv("lgbm_task_1_submission.csv", index=False)